# Task

Using the dataset with raw data containing over 31,000 medals (summer.csv) and the official Medal Tables for the Editions 1996 and 1976 from Wikipedia. (wik_1996.csv, wik_1976.csv). Using the two official Medal Tables as a reference to check for the correct output.

The goal is to minimize the divergence between the aggregated Medal Tables and the official Medal Tables. 

In [1]:
import pandas as pd     #importing pandas and numpy library for preparing the data 
import numpy as np                            

In [2]:
summer = pd.read_csv('summer.csv')      #reading the raw dataset and the official medal tables from wikipedia
wik_1996 = pd.read_csv('wik_1996.csv')
wik_1976 = pd.read_csv('wik_1976.csv')

In [3]:
summer.head()       

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [4]:
summer.tail()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
31160,2012,London,Wrestling,Wrestling Freestyle,"JANIKOWSKI, Damian",POL,Men,Wg 84 KG,Bronze
31161,2012,London,Wrestling,Wrestling Freestyle,"REZAEI, Ghasem Gholamreza",IRI,Men,Wg 96 KG,Gold
31162,2012,London,Wrestling,Wrestling Freestyle,"TOTROV, Rustam",RUS,Men,Wg 96 KG,Silver
31163,2012,London,Wrestling,Wrestling Freestyle,"ALEKSANYAN, Artur",ARM,Men,Wg 96 KG,Bronze
31164,2012,London,Wrestling,Wrestling Freestyle,"LIDBERG, Jimmy",SWE,Men,Wg 96 KG,Bronze


In [5]:
summer[summer.isna().any(axis = 1)]    #checking for null values across the columns

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
29603,2012,London,Athletics,Athletics,Pending,NaN,Women,1500M,Gold
31072,2012,London,Weightlifting,Weightlifting,Pending,NaN,Women,63KG,Gold
31091,2012,London,Weightlifting,Weightlifting,Pending,NaN,Men,94KG,Silver
31110,2012,London,Wrestling,Wrestling Freestyle,"KUDUKHOV, Besik",NaN,Men,Wf 60 KG,Silver


In [6]:
summer.dropna(inplace = True)   #dropping the null values

In [7]:
wik_1976.head()

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Soviet Union (URS),49,41,35,125
1,2,East Germany (GDR),40,25,25,90
2,3,United States (USA),34,35,25,94
3,4,West Germany (FRG),10,12,17,39
4,5,Japan (JPN),9,6,10,25


In [8]:
wik_1996.head()

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,United States (USA)*,44,32,25,101
1,2,Russia (RUS),26,21,16,63
2,3,Germany (GER),20,18,27,65
3,4,China (CHN),16,22,12,50
4,5,France (FRA),15,7,15,37


In [9]:
wik_1976.NOC.str.split('(', expand = True).iloc[:, 1].str.replace(')', '').str.replace('*','')

0         URS
1         GDR
2         USA
3         FRG
4         JPN
5         POL
6         BUL
7         CUB
8         ROU
9         HUN
10        FIN
11        SWE
12        GBR
13        ITA
14        FRA
15        YUG
16        TCH
17        NZL
18        KOR
19        SUI
20        JAM
21        PRK
22        NOR
23        DEN
24        MEX
25        TRI
26        CAN
27        BEL
28        NED
29        POR
30        ESP
31        AUS
32        IRI
33        MGL
34        VEN
35        BRA
36        AUT
37        BER
38        PAK
39        PUR
40        THA
41    41 NOCs
Name: 1, dtype: object

In [10]:
wik_1976['Country'] = wik_1976.NOC.str.split('(', expand = True).iloc[:, 1].str.replace(')', '').str.replace('*','')

In [11]:
wik_1976 = wik_1976.drop(columns = ['Rank','NOC','Total']).set_index('Country')
wik_1976.head()

,Gold,Silver,Bronze
Country,,,
URS,49,41,35
GDR,40,25,25
USA,34,35,25
FRG,10,12,17
JPN,9,6,10


In [12]:
wik_1996['Country'] = wik_1996.Nation.str.split('(', expand = True).iloc[:,1].str.replace(')', '').str.replace('*','')
wik_1996 = wik_1996.drop(columns = ['Rank','Nation','Total']).set_index('Country')
wik_1996.head()

,Gold,Silver,Bronze
Country,,,
USA,44,32,25
RUS,26,21,16
GER,20,18,27
CHN,16,22,12
FRA,15,7,15


Creating the column Event_Gender and identifying the mixed gender events(inputs from the experts)

All the events containg mixed,pairs,equestrian and sailing (before 1988) in the column are considered mixed events.

In [13]:
summer['Event_Gender'] = summer.Gender
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Men
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Men
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Men
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,Men
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,Men


In [14]:
summer.Event.str.lower().str.contains('mixed').sum()

38

In [15]:
summer.Event.str.lower().str.contains('pairs').sum()

12

In [16]:
summer.Event.str.lower().str.contains('equestrian').sum()

0

In [17]:
((summer.Sport.str.lower().str.contains('sailing')) & (summer.Year < 1988)).sum()

755

In [18]:
mask1 = summer.Event.str.lower().str.contains('mixed')
mask2 = summer.Event.str.lower().str.contains('pairs')
mask3 = summer.Sport.str.lower().str.contains('equestrian')
mask4 = ((summer.Sport.str.lower().str.contains('sailing')) & (summer.Year < 1988)).sum()

In [19]:
summer.loc[mask1 | mask2 | mask3 | mask4 , 'Event_Gender'] = 'X'
summer.Event_Gender.value_counts()

X    31161
Name: Event_Gender, dtype: int64

Medals awarded in Badminton mixed Double Events

In [20]:
badm_mixed = [21773, 21782, 21776,21785, 21770, 21779,23703,23712,23706, 23715,23709,23700,25720,25729,25723,25732,25726,
              25717,27727,27736, 27730,27739,27724,27733, 29784, 29785,29786,29787,29788,29789]

In [21]:
summer.loc[badm_mixed, "Event_Gender"] = "X"

# Identifying unique events

In [23]:
summer["Event_Medals"] = summer.groupby(["Year", "Sport", "Discipline", "Event", "Event_Gender"]).Medal.transform("count")

In [24]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medals
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,X,2
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,X,2
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,X,3
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,X,3
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,X,3


In [25]:
summer.Event_Medals.value_counts().sort_index()

1         3
2        44
3      5502
4      1564
5        40
       ... 
98      196
104     104
108     216
110     110
116     116
Name: Event_Medals, Length: 78, dtype: int64

In [26]:
summer.loc[summer.Event_Medals == 5]

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medals
1273,1908,London,Athletics,Athletics,"ARCHIBALD, Edward Blake",CAN,Men,Pole Vault,Bronze,X,5
1274,1908,London,Athletics,Athletics,"JACOBS, Charles Sherman",USA,Men,Pole Vault,Bronze,X,5
1275,1908,London,Athletics,Athletics,"SÖDERSTRÖM, Bruno",SWE,Men,Pole Vault,Bronze,X,5
1276,1908,London,Athletics,Athletics,"COOKE, Edward Tiffin",USA,Men,Pole Vault,Gold,X,5
1277,1908,London,Athletics,Athletics,"GILBERT, Alfred Carleten",USA,Men,Pole Vault,Gold,X,5
7770,1948,London,Gymnastics,Artistic G.,"MOGYOROSI-KLENCS, Janos",HUN,Men,Vault,Bronze,X,5
7771,1948,London,Gymnastics,Artistic G.,"PATAKI, Ferenc",HUN,Men,Vault,Bronze,X,5
7772,1948,London,Gymnastics,Artistic G.,"SOTORNIK, Leo",TCH,Men,Vault,Bronze,X,5
7773,1948,London,Gymnastics,Artistic G.,"AALTONEN, Paavo Johannes",FIN,Men,Vault,Gold,X,5
7774,1948,London,Gymnastics,Artistic G.,"ROVE, Olavi Antero",FIN,Men,Vault,Silver,X,5


# Identifying Team Events

All medals / rows, that were awarded in Events with more than 5 medals, shall be deemed Team Event Medals. (new column "Team")

In [28]:
summer["Team"] = pd.Series(np.where(summer.Event_Medals > 5, "Yes", "No"))

In [29]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medals,Team
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,X,2,No
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,X,2,No
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,X,3,No
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,X,3,No
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,X,3,No


In [30]:
summer.Team.value_counts()

Yes    24008
No      7149
Name: Team, dtype: int64

In [32]:
#Removing duplicated Medals in Team Events
summer.reset_index(inplace=True)
summer.head()

,level_0,index,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medals,Team
0,0,0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,X,2,No
1,1,1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,X,2,No
2,2,2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,X,3,No
3,3,3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,X,3,No
4,4,4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,X,3,No


In [33]:
singles = summer.loc[summer.Team == "No"].copy()
singles.shape

(7149, 14)

In [34]:
team = summer.loc[summer.Team == "Yes"].copy()
team.shape

(24008, 14)

In [35]:
team.drop_duplicates(subset = ["Year", "Sport", "Discipline", "Country", "Event", "Event_Gender", "Medal"], inplace = True)

In [36]:
team.shape

(7051, 14)

In [37]:
pd.concat([singles, team]).shape

(14200, 14)

In [38]:
summer_new = pd.concat([singles, team])

In [39]:
summer_new.set_index("index", inplace= True)

In [40]:
summer_new.head()

,level_0,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medals,Team
index,,,,,,,,,,,,,
0,0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,X,2,No
1,1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,X,2,No
2,2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,X,3,No
3,3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,X,3,No
4,4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,X,3,No


In [41]:
summer_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14200 entries, 0 to 31105
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   level_0       14200 non-null  int64 
 1   Year          14200 non-null  int64 
 2   City          14200 non-null  object
 3   Sport         14200 non-null  object
 4   Discipline    14200 non-null  object
 5   Athlete       14200 non-null  object
 6   Country       14200 non-null  object
 7   Gender        14200 non-null  object
 8   Event         14200 non-null  object
 9   Medal         14200 non-null  object
 10  Event_Gender  14200 non-null  object
 11  Event_Medals  14200 non-null  int64 
 12  Team          14200 non-null  object
dtypes: int64(3), object(10)
memory usage: 1.5+ MB


In [42]:
#Creating Official Medal Table
medal_tables = summer_new.groupby(["Year", "Country", "Medal"]).Medal.count().unstack(fill_value = 0)[["Gold", "Silver", "Bronze"]]
medal_tables

Medal         Gold  Silver  Bronze
Year Country                      
1896 AUS         2       0       0
     AUT         2       1       2
     DEN         1       2       3
     FRA         5       4       2
     GBR         2       3       2
...            ...     ...     ...
2012 UGA         1       0       0
     UKR         6       4       9
     USA        38      26      27
     UZB         1       0       2
     VEN         1       0       0

[1157 rows x 3 columns]

# Comparing with Wikipedia Medal Tables

In [43]:
medal_tables.head()

Medal         Gold  Silver  Bronze
Year Country                      
1896 AUS         2       0       0
     AUT         2       1       2
     DEN         1       2       3
     FRA         5       4       2
     GBR         2       3       2

In [44]:
agg_1976 = medal_tables.loc[1976].sort_values(["Gold", "Silver", "Bronze"], ascending = False).copy()

In [45]:
agg_1976.head()

Medal,Gold,Silver,Bronze
Country,,,
URS,46,38,33
GDR,39,25,24
USA,32,32,24
FRG,10,12,17
JPN,9,6,10


In [46]:
wik_1976.head()

,Gold,Silver,Bronze
Country,,,
URS,49,41,35
GDR,40,25,25
USA,34,35,25
FRG,10,12,17
JPN,9,6,10


In [48]:
div_76 = agg_1976.sub(wik_1976).abs().dropna()
div_76

Medal,Gold,Silver,Bronze
Country,,,
AUS,0.0,0.0,0.0
AUT,0.0,0.0,0.0
BEL,0.0,0.0,0.0
BER,0.0,0.0,0.0
BRA,0.0,0.0,0.0
BUL,0.0,0.0,0.0
CAN,0.0,0.0,0.0
CUB,0.0,0.0,0.0
DEN,0.0,0.0,0.0


In [49]:
score_76 = div_76.sum().sum()
score_76

16.0

In [50]:
agg_1996 = medal_tables.loc[1996].sort_values(["Gold", "Silver", "Bronze"], ascending = False).copy()
agg_1996.head()

Medal,Gold,Silver,Bronze
Country,,,
USA,37,29,23
RUS,26,21,16
GER,18,17,26
FRA,15,6,15
CHN,13,21,11


In [51]:
wik_1996.head()

,Gold,Silver,Bronze
Country,,,
USA,44,32,25
RUS,26,21,16
GER,20,18,27
CHN,16,22,12
FRA,15,7,15


In [52]:
div_96 = agg_1996.sub(wik_1996).abs().dropna()
div_96

Medal,Gold,Silver,Bronze
Country,,,
ALG,0.0,0.0,0.0
ARG,0.0,0.0,0.0
ARM,0.0,0.0,0.0
AUS,0.0,0.0,3.0
AUT,0.0,0.0,0.0
...,...,...,...
UKR,0.0,0.0,0.0
USA,7.0,3.0,2.0
UZB,0.0,0.0,0.0


In [53]:
score_96 = div_96.sum().sum()
score_96

34.0

In [54]:
print(score_76, score_96)

16.0 34.0
